<a href="https://colab.research.google.com/github/j03m/lstm-price-predictor/blob/main/Coin_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Current Todo:


Some ideas here: 

* Use pandas_ta to attach technical data to timeseries then use random forest to see if any features are super good.

* Use vector_bt to back test your models

* Integrate the different time granularity models, 15m etc into the core configs

* Fix coinbase - we can't filter on volume amount without extra work

* Add TA fields, use random forest to check which fields are the best, verify with mean square error (this could take a while)

* Move to an "always" on model where we more closely monitor exits and entries. We can constantly check predictions and refine entry/exit points. Can we pick a better entry point? We had at some point discussed trying to predict all 4 values which would help. 

* Can we monitor and alarm/email the 15 min chart of an entry and see if it is
move toward or away from our target?

* pandas_ta strategy/back testers look interesting?

Read me: https://www.kaggle.com/code/vuhuyduongnia/vn30-stock-prediction-by-lstm-model-accuracy-90


#IMPORT DATASETS AND LIBRARIES


In [16]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import sys

sys.path.insert(0,'/content/drive/My Drive/ml-trde-notebooks')

model_version_token = "2"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Library

In [ ]:
# ***** WARNING : Install deps - This will BUILD TALib and takes a while!
%run -i '/content/drive/My Drive/ml-trde-notebooks/installs.ipynb'

In [84]:
# Executes this notebook in our space, making all of its functions/globals available
%run -i '/content/drive/My Drive/ml-trde-notebooks/common.ipynb'

Modified
1673903579
1673903579.0
1673903579.0


In [17]:
models = {}

# Data Loader

In [5]:
if train_models:
  #we don't need 15 right now because they get loaded in the train, that will prob need to be more sophisticated soon
  #all_stock_df_15 = []
  for ticker in tickers:
    print("load:", ticker)
    df = sort_date(pd.read_csv(data_path +'/' + ticker + '.csv'))
    all_stock_dfs.append(df)
    #df2 = sort_date(pd.read_csv(data_path + '/' + ticker + '-15.csv').rename(columns={"Datetime":"Date"}))

  for coin in coins:
    print("load:", coin)
    df = sort_date(pd.read_csv(data_path +'/' + coin + '.csv'))
    all_coin_dfs.append(df)
else:
  print("not training/not loading")


not training/not loading


# Training/Loading

In [6]:
build_and_stash_all_config_models()

build model: lstm_cv  features: 2
build model: lstm_att_cv  features: 2
build model: lstm_cv_rvi  features: 3
build model: svm_cv  features: 2
build model: lstm_xgb_cols  features: 9


## Load all models from disk

In [18]:
if load_models:
  training_filter = model_config["training_filter"]
  for name in model_config["day_bar_models"]:
    if (name in training_filter): #skip training models
      print("skipping: ", name)
      continue
    print("loading:", name)
    if name in model_config["load_type"] and model_config["load_type"][name] == "joblib":
      models[name] = joblib.load(model_path + "/" + name  + "-" + model_version_token + ".joblib") 
    else:
      models[name] = keras.models.load_model(model_path + "/" + name + "-" + model_version_token + ".h15")
  lstm_15m = keras.models.load_model(model_path + "/lstm_15m.h15")
  print ("models loaded")  

else:
  print ("Not loading")
  


loading: lstm_cv
loading: lstm_att_cv
loading: lstm_cv_rvi
loading: svm_cv
loading: lstm_xgb_cols
models loaded


## Train models


In [8]:
histories = {}
training_filter = model_config["training_filter"]
if train_models:
  for name in model_config["day_bar_models"]:
    if (len(training_filter)!=0 and not name in training_filter):
      print("skipping: ", name)
      continue
    build_model_from_config(models, name, model_config)
    print("training: ", name)
    # todo - we need a training set type flag in config for 15m models to get the correct files/data
    training_dfs = get_training_datasets_for_model(name)
    print("training dfs:", len(training_dfs))
    count = 0
    for df in training_dfs:
      print("training: ", name, " df: ", count)
      count+=1
      history = train_config_model_against_df(name, model_config, df)
      if not name in histories:
        histories[name] = []
      histories[name].append(history)
  lstm_15m = build_15m_model()

## Train a single model against a single dataset

In [9]:
#build_and_stash_all_config_models()
#coin_dfs = get_raw_data_for_coin_list(180)

In [10]:
#[model, history] = train_config_model_against_df("lstm_cv", model_config, coin_dfs["FCON-USDT"])
#models["lstm_cv"] = model
#df_raw = get_coin_data_frames(180, "FCON-USDT")
#test_name = "lstm_ohlc"
#all_columns = model_config["column_sets"][test_name]+gbl_target_column
#[scaled_features, X, y, normal_features] = convert_to_training_dataset(df, columns=all_columns)  
#[p_scaled, p] = predict_trade(models[test_name], test_name, "FCON-USDT", X, all_columns)
#build_trade_model(p, normal_features, "FCON-USDT", test_name)

# Save all models to disk

In [11]:
if save_models:
  for name in model_config["day_bar_models"]:
    if (len(training_filter)!=0 and not name in training_filter):
      print("skipping: ", name)
      continue
    print("saving:", name)
    if name in model_config["load_type"] and model_config["load_type"][name] == "joblib":
      joblib.dump(models[name], model_path + "/" + name  + "-" + model_version_token + ".joblib") 
    else:
      models[name].save(model_path + "/" + name  + "-" + model_version_token + ".h15")
  lstm_15m.save(model_path + "/lstm_15m-2.h15")
  print ("models saved")  
else:
  print ("Not saving")


Not saving


# Visualize and Compare all Models

In [86]:
df_raw = get_coin_data_frames(30, "DOGE3L-USDT")

In [87]:
#data
[scaled_features, X, y, normal_features] = convert_to_training_dataset(df_raw, columns=gbl_all_features)  

df_chart = pd.DataFrame();
normal_features.reset_index(inplace=True)
df_chart["Date"] = normal_features["Date"]
scaled_close = pluck(X, gbl_all_features, ["Close"])
df_chart["Close"] = scaled_close
graph_columns = ["Date","Close"]

results = pd.DataFrame();

stuff = models.items()

#backtest_filter = model_config["backtest_filter"]
#backtest_filter = ["lstm_cv", "lstm_cv_rvi", "lstm_att_cv", "svm_cv", "lstm_xgb_cols"]
backtest_filter = []
for key, model in stuff:
  
  print("key:", key)
  if (len(backtest_filter)!=0 and not key in backtest_filter):
    print("skipping:", key)
    continue

  print("testing:", key)
  graph_columns.append(key)
  
  columns = model_config["column_sets"][key]
  
  x = pluck(X, gbl_all_features, columns)

  predicted = model.predict(x).flatten()

  mse = mean_squared_error(scaled_close, predicted)
  print("predicted mse for model: ", key, mse)
  
  [df_profit, profit] = build_profit_estimate(predicted, normal_features)
  print("profit for model: ", key, profit)

  results = results.append([[key, mse, profit]])

  df_chart[key] = predicted

interactive_plot(df_chart[graph_columns], "wow") 
results

key: lstm_cv
testing: lstm_cv
1/1 [==============================] - 0s 27ms/step
predicted mse for model:  lstm_cv 0.00017016485715844567
profit for model:  lstm_cv 5744.500532313687
key: lstm_att_cv
testing: lstm_att_cv
1/1 [==============================] - 0s 34ms/step
predicted mse for model:  lstm_att_cv 0.0008546609715172177
profit for model:  lstm_att_cv 5744.500532313687
key: lstm_cv_rvi
testing: lstm_cv_rvi
1/1 [==============================] - 0s 42ms/step
predicted mse for model:  lstm_cv_rvi 0.00034420904290522435
profit for model:  lstm_cv_rvi 5744.500532313687
key: svm_cv
testing: svm_cv
1/1 [==============================] - 0s 38ms/step
predicted mse for model:  svm_cv 0.00027651792699803367
profit for model:  svm_cv 5744.500532313687
key: lstm_xgb_cols
testing: lstm_xgb_cols
1/1 [==============================] - 0s 66ms/step
predicted mse for model:  lstm_xgb_cols 0.004774967095605961
profit for model:  lstm_xgb_cols 3726.980987227878


,0,1,2
0,lstm_cv,0.000170,5744.500532
0,lstm_att_cv,0.000855,5744.500532
0,lstm_cv_rvi,0.000344,5744.500532
0,svm_cv,0.000277,5744.500532
0,lstm_xgb_cols,0.004775,3726.980987


In [14]:
df_profit

,Date,Predicted,Predicted-Target,Predicted-Diff,Should-Trade,RealDiff,Percent,Profit
0,2022-09-18,0.149880,0.157932,0.008052,True,0.000054,0.051185,255.924171
1,2022-09-19,0.157932,0.152560,-0.005371,False,-0.000009,-0.008115,0.000000
2,2022-09-20,0.152560,0.157174,0.004613,True,-0.000071,-0.064545,-322.727273
3,2022-09-21,0.157174,0.153480,-0.003694,False,0.000143,0.138970,0.000000
4,2022-09-22,0.153480,0.131911,-0.021569,False,0.000207,0.176621,0.000000
...,...,...,...,...,...,...,...,...
115,2023-01-11,0.138162,0.141692,0.003530,True,0.000034,0.064762,323.809524
116,2023-01-12,0.141692,0.145500,0.003808,True,0.000077,0.137746,688.729875
117,2023-01-13,0.145500,0.159219,0.013719,True,0.000071,0.111635,558.176101
118,2023-01-14,0.159219,0.148826,-0.010393,False,-0.000032,-0.045262,0.000000


# What has a buy indicator for tomorrow?

In [88]:
# Fetch the top 10 and see if they predict up
df_products = get_all_products()
df_products = df_products[df_products.id.str.endswith('USDT')]

# we have a desired budget of 10k in trading capital to deploy. 
# "volValue": is the 24h total, the trading volume in quote currency of last 24 hours
# Any markets we enter need to have a signifcantly higher trading value volume otherwise
# we can't really trade there without breaking things up. As we grow our strat here will need to change.
# for now we cut down our set to 1m in volVal
volumeCutoff = 500000
df_products["volValue"] = [float(x) for x in df_products['volValue']]
df_products = df_products[df_products["volValue"] > volumeCutoff]
df_products = df_products.sort_values(by = ['id'])

models_in_play = [
        "lstm_cv", 
        "lstm_att_cv", 
        "lstm_cv_rvi",
        "svm_cv"]
if coin_base:
  df_products = df_products[df_products.trading_disabled == False]
  df_products = df_products[df_products.cancel_only == False]

df_trades = pd.DataFrame();
df_estc = pd.DataFrame(); #expected short term closes
df_estc["Product"] = [];
df_estc["Est Close"] = [];
df_estc["Est Close Raw"] = [];
bars_long = 180
bars_short = 30
counter = 0;
print("iterating through:", len(df_products))
tries = 3

def downloadAndPredict(all_trades, product, length):
  print("download day bars: ", product, " bar set:", length)
  df_raw = get_coin_data_frames(length, product)
  for name in models_in_play:
    print("predicting trade:", name, "for", product)
    df_trade = predict_config_model_for_product(df_raw, name, product)
    df_trade["Period"] = [length]
    all_trades = all_trades.append(df_trade)
  return all_trades

def downloadAndPredict15(all_trades, product):
  print("download 15 bars: ", product)
  [predicted_scaled, predicted] = fetch_and_predict_short_term(lstm_15m,product)
  df2 = pd.DataFrame({'Product': [product], 'Est Close': [predicted], 'Est Close Raw': predicted_scaled})
  all_trades = all_trades.append(df2)
  return all_trades


for index, row in df_products.iterrows():
    
    loop = True
    count = 0
    while(loop):
      try:
        print("start day long")
        df_trades = downloadAndPredict(df_trades, row.id, bars_long)
        time.sleep(1)
        
        print("start day short")
        df_trades = downloadAndPredict(df_trades, row.id, bars_short)
        time.sleep(1)

        print("start 15m")
        df_estc = downloadAndPredict15(df_estc, row.id)
        time.sleep(1)
        loop = False
      except Exception as inst:
        raise inst
        print("Error: ", inst)
        time.sleep(1)
        count = count+1
        if count>tries:
          loop = False

df_trades.reset_index()
df_trades_filtered = df_trades.loc[(df_trades["Period"] == 180) & (df_trades["MSE"] < 0.0005) | (df_trades["Period"] == 30) & (df_trades["MSE"] < 0.005)]
df_buys = df_trades_filtered[df_trades_filtered['Move'] > 0] 
df_shorts = df_trades_filtered[df_trades_filtered['Move'] < 0] 
df_weighted = df_trades_filtered.groupby("Product").apply(consensus_percent)
df_weighted = df_weighted.rename("Consensus Percent")
df_trades_final = pd.merge(df_trades_filtered, df_weighted, left_on='Product', right_index=True)
df_weighted = df_trades_filtered.groupby("Product").apply(consensus_prediction)
df_weighted = df_weighted.rename("Consensus Prediction")
df_trades_final = pd.merge(df_trades_final, df_weighted, left_on='Product', right_index=True)
df_view = df_trades_final[["Product",  "Model Name",  "MSE", "Period", "Close", "Predicted", "Percent","Consensus Percent", "Consensus Prediction"]]
df_view = df_trades_final.sort_values(by=["Consensus Percent", "MSE"], ascending=[True, False])

iterating through: 121
start day long
download day bars:  1EARTH-USDT  bar set: 180
predicting trade: lstm_cv for 1EARTH-USDT
6/6 [==============================] - 0s 12ms/step
predicting trade: lstm_att_cv for 1EARTH-USDT
6/6 [==============================] - 0s 12ms/step
predicting trade: lstm_cv_rvi for 1EARTH-USDT
6/6 [==============================] - 0s 16ms/step
predicting trade: svm_cv for 1EARTH-USDT
6/6 [==============================] - 0s 9ms/step
start day short
download day bars:  1EARTH-USDT  bar set: 30
predicting trade: lstm_cv for 1EARTH-USDT
1/1 [==============================] - 0s 25ms/step
predicting trade: lstm_att_cv for 1EARTH-USDT
1/1 [==============================] - 0s 24ms/step
predicting trade: lstm_cv_rvi for 1EARTH-USDT
1/1 [==============================] - 0s 31ms/step
predicting trade: svm_cv for 1EARTH-USDT
1/1 [==============================] - 0s 30ms/step
start 15m
download 15 bars:  1EARTH-USDT
1/1 [==============================] - 1s 1s/step

Exception: ignored

In [ ]:
df_trades_filtered = df_trades.loc[(df_trades["Period"] == 180) & (df_trades["MSE"] < 0.0005) | (df_trades["Period"] == 30) & (df_trades["MSE"] < 0.002)]
df_buys = df_trades_filtered[df_trades_filtered['Move'] > 0] 
df_shorts = df_trades_filtered[df_trades_filtered['Move'] < 0] 

df_weighted = df_trades_filtered.groupby("Product").apply(consensus_percent)
df_weighted = df_weighted.rename("Consensus Percent")
df_trades_final = pd.merge(df_trades_filtered, df_weighted, left_on='Product', right_index=True)

df_weighted = df_trades_filtered.groupby("Product").apply(consensus_prediction)
df_weighted = df_weighted.rename("Consensus Prediction")
df_trades_final = pd.merge(df_trades_final, df_weighted, left_on='Product', right_index=True)

df_weighted = df_trades_filtered.groupby("Product").apply(consensus_overall)
df_weighted = df_weighted.rename("Consensus Score")
df_trades_final = pd.merge(df_trades_final, df_weighted, left_on='Product', right_index=True)


df_view = df_trades_final[["Product",  "Model Name",  "MSE", "Period", "Close", "Predicted", "Percent","Consensus Percent", "Consensus Prediction", "Consensus Score"]]
df_view = df_view.sort_values(by=["Consensus Percent", "MSE"], ascending=[False, True])

df_view

In [ ]:
df_estc

In [ ]:
df_trades[df_trades["Product"] == "XDB-USDT"]

In [ ]:
now = datetime.now()
today = now.strftime("%Y-%m-%d")

if coin_base:
  token = "cb"
else:
  token = "ku"

df_buys.to_csv(data_path+"/buy-" + token + "-" + today + ".csv")
df_estc.to_csv(data_path+"/15m-" + token + "-" + today + ".csv")
df_shorts.to_csv(data_path+"/shorts-" + token + "-" + today + ".csv")
df_trades_final.to_csv(data_path+"/final-" + token + "-" + today + ".csv")

In [ ]:
df_estc